# Trabajo Práctico 1: Análisis Exploratorio de Datos #
## Cientes potenciales a tarjetas de crédito. ##
El [DATASET](https://www.kaggle.com/datasets/sajidhussain3/jobathon-may-2021-credit-card-lead-prediction?select=train.csv) contiene información de clientes de un banco denominado Happy Customer Bank. Es un banco privado de tamaño medio que se ocupa de todo tipo de productos bancarios, como cuentas de ahorro, cuentas corrientes, productos de inversión, productos de crédito, entre otras ofertas.  

En este caso, el Happy Customer Bank desea realizar ventas cruzadas de sus tarjetas de crédito a sus clientes existentes. El banco ha identificado un conjunto de clientes que son elegibles para aceptar estas tarjetas de crédito.

El banco tiene la necesidad, de identificar a los clientes que podrían mostrar intención hacia una tarjeta de crédito recomendada.

<img src = "https://storage.googleapis.com/kaggle-datasets-images/1371711/2277422/2c35585d42b3747882e83e926d0f643e/dataset-cover.png?t=2021-05-27-18-40-42">


 --- 

## 1. Listado de variables y selección ###
### a) Variables de Entrada ##

- **ID:** Identificador único para cada fila. (no lo vamos a usar)



- **Gender:** Género del cliente.



- **Age:** Edad del cliente (en años).


        
- **Region_Code:** Código de la Región para los clientes


        
- **Occupation:** Ocupación del cliente.

        
        
- **Channel_Code:** Código de Canal de Adquisición para el Cliente (Codificado)


        
- **Vintage:** Antiguedad del Cliente (En Meses).

        
        
- **Credit_Product:** Si el Cliente tiene algún producto de crédito activo (Préstamo de vivienda, Préstamo personal, Tarjeta de crédito, etc.)



- **Avg_Account_Balance:** Saldo de cuenta promedio. Saldo promedio de la cuenta del cliente en los últimos 12 meses

      
      
- **Is_Active:** Si el Cliente tiene activo algún producto de crédito. Si el cliente está activo en los últimos 3 meses. 
            0-> El cliente no esta activo | 1-> el cliente esta activo
            
            
--------

### b) Variable de Salida ###

- **Is_Lead:**  Si el Cliente está interesado en la Tarjeta de Crédito.
            0-> El cliente no esta interesado | 1-> el cliente esta interesado



In [ ]:
import pandas as pd 
import pytz

In [ ]:
passes = pd.read_csv('train.csv')

In [ ]:
passes.sample(10)

### b) Transformaciones ###

- **Gender:** one-hot encoder



- **Age:** rango


        
- **Region_Code:** one-hot encoder


        
- **Occupation:** one-hot encoder

        
        
- **Channel_Code:** one-hot encoder


        
- **Vintage:** rango

        
        
- **Credit_Product:** one-hot encoder y ver que hacemos con los null. (esta desbalanceado)



- **Avg_Account_Balance:** escalar y rangos (y ver outliers)

      
      
- **Is_Active:** one-hot encoder